In [1]:
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('./자료/와인/train.csv')
test = pd.read_csv('./자료/와인/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace = True)

print(train.quality.value_counts())
train.quality -= 3

6    2416
5    1788
7     924
4     186
8     152
3      26
9       5
Name: quality, dtype: int64


In [2]:
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,2,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,2,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,3,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [3]:
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [4]:
model = XGBClassifier()

In [5]:
X = train_one.drop('quality',axis= 1)
y = train_one['quality']

model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [6]:
pred = model.predict(test_one)

In [7]:
submission = pd.read_csv('./자료/와인/sample_submission.csv')
submission['quality'] = pred
submission.head()

,index,quality
0,0,2
1,1,3
2,2,3
3,3,2
4,4,3


In [8]:
submission.to_csv('xgb_pred.csv', index = False)

In [9]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

In [10]:
X = train_one.drop('quality',axis= 1)
y = train_one['quality']

model.fit(X, y)

LGBMClassifier()

In [11]:
pred = model.predict(test_one)

In [12]:
submission = pd.read_csv('./자료/와인/sample_submission.csv')
submission['quality'] = pred
submission.head()

,index,quality
0,0,2
1,1,2
2,2,3
3,3,2
4,4,3


In [13]:
submission.to_csv('lgbm_pred.csv', index = False)

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits = 5)

cnt = 1
acc = 0
for train_idx, valid_idx in skf.split(X,y):
  
  train_data = train_one.iloc[train_idx]
  valid_data = train_one.iloc[valid_idx]

  model = LGBMClassifier()

  train_X = train_data.drop('quality',axis= 1)
  train_y = train_data['quality']

  model.fit(train_X,train_y)


  valid_X = valid_data.drop('quality',axis = 1)
  valid_y = valid_data['quality']

  pred = model.predict(valid_X)

  print(cnt," 번째 모델 정확도 : " ,accuracy_score(pred,valid_y))
  acc += accuracy_score(pred,valid_y)
  cnt+=1

print('모델 정확도 평균 : ',acc/5)

1  번째 모델 정확도 :  0.6609090909090909
2  번째 모델 정확도 :  0.6581818181818182
3  번째 모델 정확도 :  0.6606005459508644
4  번째 모델 정확도 :  0.632393084622384
5  번째 모델 정확도 :  0.6469517743403094
모델 정확도 평균 :  0.6518072628008933
